## Formatting the climate data
So it has named dimensions if they don't already

In [37]:
# Libraries
%load_ext autoreload
%autoreload

import numpy as np
import iris
import iris.coord_categorisation
import pandas as pd

import sys
sys.path.append('../')
from libs.plot_maps import *


import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import numpy.ma as ma
%matplotlib inline
import cartopy.crs as ccrs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading in the data (ignoring precipation because (a) we don't need it and (b) it has additional dimension coordinates (lat_bnds and lon_bnds) which upset iris.

In [80]:
dir = '../data/climate/'
files      = { 'air'               : 'air.mon.mean.nc',
               'lightning'         : 'lightn_climatology_otd_mlnha.nc',
               'relative_humidity' : 'rhum.mon.mean.nc'}

input_data = {}

for key, f in files.items():
    print(key)
    dat = iris.load_cube(dir + f)
    input_data[key] = dat

air
lightning
relative_humidity


Adding a year coordinate to relative humidity and air files and cutting it off before 2001:

In [81]:
for key in input_data.keys():
    if key != 'lightning':
            # Add coord of year
            iris.coord_categorisation.add_year(input_data[key], 'time', name='year')
            
            # Extract from 2001 onwards
            post_2001[key] = input_data[key].extract(iris.Constraint(year=lambda y: y >= 2001))

Adding a coord system to the files, because otherwise they aren't be regridded

Regrid lightning to the mask:

In [87]:
air_temp = input_data['air'].collapsed('time', iris.analysis.MEAN)

input_data['lightning'] = input_data['lightning'].regrid(air_temp, iris.analysis.Linear())

In [92]:
lightning = post_2001['air'].copy()
print(input_data['lightning'])
print(lightning)

lightn / (1)                        (time: 12; latitude: 73; longitude: 144)
     Dimension coordinates:
          time                           x             -              -
          latitude                       -             x              -
          longitude                      -             -              x
     Attributes:
          comment_1: LIS/OTD 0.5 Degree High Resolution Monthly Climatology (HRMC)
          comment_2:: non-transient
          comment_3:: (i.e, average lightening in period of oberservation for each month)
          comment_4:: Units: Flash count per km2
          history: Wed Apr  2 19:04:33 2008: ncap -O -s lightn[time,lat,lon]=float(lightn*10000.)...
Monthly Mean Air Temperature at sigma level 0.995 / (degC) (time: 224; latitude: 73; longitude: 144)
     Dimension coordinates:
          time                                                  x              -              -
          latitude                                              -             

In [94]:
lightning = post_2001['air'].copy()
light = lightning.aggregated_by(['year'], iris.analysis.MEAN)
j = 0
for k in range(len(light.coord('year').points) - 1):
    lightning.data[j:j+12,:,:] = input_data['lightning'].data
    print('j: ', j)

# In 2019, we've only reached Aug, so I'll add the data in for this single case
lightning.data[j:,:,:] = input_data[key][:8].data
post_2001['lightning'] = lightning

2001
j:  12
2002
j:  24
2003
j:  36
2004
j:  48
2005
j:  60
2006
j:  72
2007
j:  84
2008
j:  96
2009
j:  108
2010
j:  120
2011
j:  132
2012
j:  144
2013
j:  156
2014
j:  168
2015
j:  180
2016
j:  192
2017
j:  204
2018
j:  216


In [97]:
for key in post_2001.keys():
    out = key + '2001-2019.nc'
    iris.save(post_2001[key], dir + out)

Monthly Mean Air Temperature at sigma level 0.995 / (degC) (time: 224; latitude: 73; longitude: 144)
     Dimension coordinates:
          time                                                  x              -              -
          latitude                                              -              x              -
          longitude                                             -              -              x
     Auxiliary coordinates:
          year                                                  x              -              -
     Attributes:
          Conventions: COARDS
          NCO: 20121012
          References: http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis.derived...
          actual_range: [-73.78001  42.14595]
          dataset: NCEP Reanalysis Derived Products
          dataset_title: NCEP-NCAR Reanalysis 1
          description: Data from NCEP initialized reanalysis (4x/day).  These are the 0.9950 sigma...
          history: Thu May  4 20:11:16 2000: